# Multiverse Math

In this task, the agent is operating in an alternate universe which in which the basic mathematical operations like addition and multiplication are different.

The agent must use tools that allow is to carry out calculations in this universe.

This task can help verify that an agent is able to ignore its own knowledge of math and instead correctly use information returned by the tools.

The modified mathematical operations yield different reuslts, but still retain some properties (e.g., the modified multiplication operation is still commutative).

Please note that the modified operations are not guaranteed to even make sense in the real world since not all properties will be retained (e.g., distributive property).

------------------

For this code to work, please configure LangSmith environment variables with your credentials.

In [1]:
import os

os.environ["LANGCHAIN_API_KEY"] = "sk-..."  # Your LangSmith API key

In [1]:
from langchain_benchmarks import clone_public_dataset, registry

In [2]:
task = registry["Multiverse Math"]
task

Name,Multiverse Math
Type,ToolUsageTask
Dataset ID,594f9f60-30a0-49bf-b075-f44beabf546a
Description,"An environment that contains a few basic math operations, but with altered results. For example, multiplication of 5*3 will be re-interpreted as 5*3*1.1. The basic operations retain some basic properties, such as commutativity, associativity, and distributivity; however, the results are different than expected. The objective of this task is to evaluate the ability to use the provided tools to solve simple math questions and ignore any innate knowledge about math."


Clone the dataset associaetd with this task

In [3]:
clone_public_dataset(task.dataset_id, dataset_name=task.name)

Dataset Multiverse Math already exists. Skipping.
You can access the dataset at https://smith.langchain.com/o/e081f11e-fbd2-41b4-9fa8-5d76c76ef854/datasets/ddca73f1-ceda-4562-8c49-7ee0a9df2a01.


## The Environment

Let's check the environment

In [4]:
env = task.create_environment()
env.tools[:5]

[StructuredTool(name='multiply', description='multiply(a: float, b: float) -> float - Multiply two numbers; a * b.', args_schema=<class 'pydantic.v1.main.multiplySchemaSchema'>, func=<function multiply at 0x7f938669fb80>),
 StructuredTool(name='add', description='add(a: float, b: float) -> float - Add two numbers; a + b.', args_schema=<class 'pydantic.v1.main.addSchemaSchema'>, func=<function add at 0x7f938669fca0>),
 StructuredTool(name='divide', description='divide(a: float, b: float) -> float - Divide two numbers; a / b.', args_schema=<class 'pydantic.v1.main.divideSchemaSchema'>, func=<function divide at 0x7f938669fc10>),
 StructuredTool(name='subtract', description='subtract(a: float, b: float) -> float - Subtract two numbers; a - b.', args_schema=<class 'pydantic.v1.main.subtractSchemaSchema'>, func=<function subtract at 0x7f938669fe50>),
 StructuredTool(name='power', description='power(a: float, b: float) -> float - Raise a number to a power; a ** b.', args_schema=<class 'pydant

Multiplying 2 x 4 = 8.8!!

In [5]:
env.tools[0].invoke({"a": 2, "b": 4})

8.8

The task instructions

In [6]:
task.instructions

'You are requested to solve math questions in an alternate mathematical universe. The operations have been altered to yield different results than expected. Do not guess the answer or rely on your  innate knowledge of math. Use the provided tools to answer the question. While associativity and commutativity apply, distributivity does not. Answer the question using the fewest possible tools. Only include the numeric response without any clarifications.'

## Agent

Let's build an agent that we can use for evaluation.

In [7]:
from langchain_benchmarks.tool_usage import agents

agent_factory = agents.OpenAIAgentFactory(task, model="gpt-3.5-turbo-16k")

# Let's test that our agent works
agent = agent_factory.create()
agent.invoke({"question": "how much is 3 + 5"})

{'question': 'how much is 3 + 5',
 'output': 'The result of 3 + 5 in this alternate mathematical universe is 9.2.',
 'intermediate_steps': [(AgentActionMessageLog(tool='add', tool_input={'a': 3, 'b': 5}, log="\nInvoking: `add` with `{'a': 3, 'b': 5}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "a": 3,\n  "b": 5\n}', 'name': 'add'}})]),
   9.2)]}

## Eval

Let's evaluate an agent now

In [8]:
from langsmith.client import Client

from langchain_benchmarks.tool_usage import STANDARD_AGENT_EVALUATOR

client = Client()

test_run = client.run_on_dataset(
    dataset_name=task.name,
    llm_or_chain_factory=agent_factory.create,
    evaluation=STANDARD_AGENT_EVALUATOR,
    verbose=True,
    tags=["gpt-3.5-turbo-16k"],
)

View the evaluation results for project 'test-weary-wing-36' at:
https://smith.langchain.com/o/e081f11e-fbd2-41b4-9fa8-5d76c76ef854/projects/p/33124759-882e-4a5c-a121-736310a40a1f?eval=true

View all tests for Dataset Multiverse Math at:
https://smith.langchain.com/o/e081f11e-fbd2-41b4-9fa8-5d76c76ef854/datasets/ddca73f1-ceda-4562-8c49-7ee0a9df2a01
[------------------------------------------------->] 10/10
 Eval quantiles:
                                    0.25       0.5      0.75      mean  \
Intermediate steps correctness  0.250000  1.000000  1.000000  0.700000   
# steps / # expected steps      1.000000  1.000000  1.000000  1.033333   
correctness                     0.000000  0.000000  1.000000  0.400000   
execution_time                  5.771554  5.771554  5.771554  5.771554   

                                    mode  
Intermediate steps correctness  1.000000  
# steps / # expected steps      1.000000  
correctness                     0.000000  
execution_time                

## Inspect

You can take a look at the underlying results.

In [9]:
import pandas as pd

df = test_run.to_dataframe()
df = pd.json_normalize(df.to_dict(orient="records"))

In [10]:
df["correctness"].mean()

0.4

In [11]:
df["num_expected_steps"] = df["reference.expected_steps"].apply(len)
df["actual_number_of_steps"] = df["output.intermediate_steps"].apply(len)

In [12]:
df.head()

,Intermediate steps correctness,# steps / # expected steps,correctness,execution_time,input.question,output.question,output.output,output.intermediate_steps,reference.reference,reference.expected_steps,num_expected_steps,actual_number_of_steps
0,1,1.0,1,5.771554,Add 2 and 3,Add 2 and 3,The sum of 2 and 3 in this alternate mathemati...,"[(tool='add' tool_input={'a': 2, 'b': 3} log=""...",6.20,[add],1,1
1,1,1.0,0,5.771554,Subtract 3 from 2,Subtract 3 from 2,The result of subtracting 3 from 2 in this alt...,"[(tool='subtract' tool_input={'a': 2, 'b': 3} ...",-4.00,[subtract],1,1
2,1,1.0,1,5.771554,What is -5 if evaluated using the negate funct...,What is -5 if evaluated using the negate funct...,The result of evaluating -5 using the negate f...,"[(tool='negate' tool_input={'a': -5} log=""\nIn...",-5.00,[negate],1,1
3,1,1.0,0,5.771554,what is the result of 2 to the power of 3?,what is the result of 2 to the power of 3?,The result of 2 to the power of 3 is 32.,"[(tool='power' tool_input={'a': 2, 'b': 3} log...",32.00,[power],1,1
4,0,1.0,0,5.771554,I ate 1 apple and 2 oranges every day for 7 da...,I ate 1 apple and 2 oranges every day for 7 da...,You ate a total of 32.34 fruits.,"[(tool='add' tool_input={'a': 1, 'b': 2} log=""...",32.34,"[multiply, add]",2,2
